In [1]:
!pip install adapter-transformers
!pip install datasets

     |████████████████████████████████| 2.5 MB 8.0 MB/s 
     |████████████████████████████████| 3.3 MB 54.1 MB/s 
     |████████████████████████████████| 50 kB 3.8 MB/s 
     |████████████████████████████████| 895 kB 54.0 MB/s 
     |████████████████████████████████| 264 kB 8.5 MB/s 
     |████████████████████████████████| 243 kB 65.2 MB/s 
     |████████████████████████████████| 119 kB 65.7 MB/s 


In [2]:
cd /content/drive/MyDrive/CAMemBERT2

/content/drive/MyDrive/CAMemBERT2


In [3]:
from transformers import AutoModelWithHeads,AutoConfig,Trainer,TrainingArguments,EarlyStoppingCallback
import numpy as np
import pdb
from sklearn.metrics import f1_score,accuracy_score,fbeta_score
from trainer_utils import get_dataset
import torch
import pandas as pd

dataset_obj,dataset_dict = get_dataset(['ged'],max_length=512)
# prepare for model
dataset_dict = dataset_dict.remove_columns(['dataset','scores'])

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/content/drive/MyDrive/CAMemBERT2/preprocessing_f.py:53: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ged = pd.DataFrame(pd.read_csv(f'data/fce-public.{self.set_type}.original.tsv',sep='  ',names=['word']).word.str.split('\t',1).tolist(),columns = ['word','labels'])
/content/drive/MyDrive/CAMemBERT2/preprocessing_f.py:53: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ged = pd.DataFrame(pd.read_csv(f'data/fce-public.{self.set_type}.original.tsv',sep='  ',names=['word']).word.str.split('\t',1).tolist(),columns = ['word','labels'])
/content/drive/MyDrive/CAMemBERT2/preprocessing_f.py:53: ParserWarning: Fa

...

  0%|          | 0/266 [00:00<?, ?ex/s]

  0%|          | 0/243 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [4]:
for lr in [1e-4]:
    for bs in [8]:
        config = AutoConfig.from_pretrained('distilroberta-base',num_labels=2)
        model = AutoModelWithHeads.from_pretrained('distilroberta-base',config=config)
        adaptors_name='ged'
        model.add_adapter(adaptors_name)
        model.add_tagging_head(
                    adaptors_name,
                    num_labels=2,
                    overwrite_ok=True
                )
        model.train_adapter(adaptors_name)
        class_weights = dataset_obj.get_weights()
        setattr(model,'class_weights',class_weights)

        def compute_metrics(p):
            logits, labels = p.predictions.flatten() , p.label_ids.flatten()
            mask = labels!=-100
            logits , labels = logits[mask] , labels[mask]
            f1 , acc , f0_5 = f1_score(logits,labels) , accuracy_score(logits,labels) , fbeta_score(logits,labels,beta=0.5)
            return {
                    "f1": f1,
                    "acc": acc,
                    "f0_5":f0_5,
                    }

        class GedWordLevelTrainer(Trainer):

            loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)

            def compute_loss(self,model,inputs,return_outputs=False):
                logits = model(**inputs).logits
                labels = inputs['labels']
                # Only keep active parts of the loss
                active_loss = inputs['attention_mask'].view(-1) == 1
                active_logits = logits.view(-1, model.config.num_labels)
                active_labels = torch.where(active_loss, labels.view(-1), torch.tensor(self.loss_fct.ignore_index).type_as(labels))
                non_padded_mask = active_labels!=-100
                loss = self.loss_fct(active_logits[non_padded_mask], active_labels[non_padded_mask])
                active_preds_mask = torch.logical_and(active_loss,non_padded_mask)
                return (loss,(loss,torch.argmax(logits,2))) if return_outputs else loss

        label_names = ['labels']
        args = TrainingArguments(
                output_dir = '/content',
                save_total_limit = 1,
                evaluation_strategy = "epoch",
                learning_rate = lr,
                per_device_train_batch_size=bs,
                per_device_eval_batch_size=bs,
                remove_unused_columns=False,
                num_train_epochs = 15,
                weight_decay = 0.01,
                save_strategy = 'epoch',
                load_best_model_at_end=True,
                metric_for_best_model='f0_5',
                label_names=label_names
            )

        trainer = GedWordLevelTrainer(
            model=model,
            args=args,
            train_dataset=dataset_dict['train'],
            eval_dataset=dataset_dict['dev'],
            compute_metrics=compute_metrics
            )

        early_stopping = EarlyStoppingCallback(early_stopping_patience = 3)
        trainer.add_callback(early_stopping)
        trainer.train()
        trainer.evaluate(dataset_dict['test'])
        pd.DataFrame(trainer.state.log_history).to_csv(f'results/gedadapters_distilroberta_lr{lr}_bs{bs}_norm.csv')

Downloading:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for prediction

Epoch,Training Loss,Validation Loss,F1,Acc,F0 5
1,No log,0.576262,0.410423,0.750235,0.524543
2,No log,0.559904,0.454031,0.809478,0.525331
3,No log,0.528174,0.470677,0.802986,0.563491
4,0.576800,0.547304,0.498418,0.840871,0.547306
5,0.576800,0.544410,0.510699,0.849401,0.551278
6,0.576800,0.536713,0.514527,0.848554,0.559662


***** Running Evaluation *****
  Num examples = 79
  Batch size = 8
Saving model checkpoint to /content/checkpoint-133
Configuration saved in /content/checkpoint-133/ged/adapter_config.json
Module weights saved in /content/checkpoint-133/ged/pytorch_adapter.bin
Configuration saved in /content/checkpoint-133/ged/head_config.json
Module weights saved in /content/checkpoint-133/ged/pytorch_model_head.bin
Configuration saved in /content/checkpoint-133/ged/head_config.json
Module weights saved in /content/checkpoint-133/ged/pytorch_model_head.bin
***** Running Evaluation *****
  Num examples = 79
  Batch size = 8
Saving model checkpoint to /content/checkpoint-266
Configuration saved in /content/checkpoint-266/ged/adapter_config.json
Module weights saved in /content/checkpoint-266/ged/pytorch_adapter.bin
Configuration saved in /content/checkpoint-266/ged/head_config.json
Module weights saved in /content/checkpoint-266/ged/pytorch_model_head.bin
Configuration saved in /content/checkpoint-266/

In [5]:
trainer.evaluate(dataset_dict['test'])

***** Running Evaluation *****
  Num examples = 97
  Batch size = 8


{'epoch': 6.0,
 'eval_acc': 0.7861494237717239,
 'eval_f0_5': 0.5640482549037665,
 'eval_f1': 0.4754511934795265,
 'eval_loss': 0.5573367476463318,
 'eval_runtime': 1.9316,
 'eval_samples_per_second': 50.217,
 'eval_steps_per_second': 6.73}

In [7]:
model.save_adapter('adapters/ged/','ged')

Configuration saved in adapters/ged/adapter_config.json
Module weights saved in adapters/ged/pytorch_adapter.bin
Configuration saved in adapters/ged/head_config.json
Module weights saved in adapters/ged/pytorch_model_head.bin
